# <center> Проект 2: Анализ вакансий на hh.ru.

## <center>Работа в реляционной базе данных 

Задача: Подготовить текстовый Google-документ, в котором по каждой встречающейся задаче нужно привести ответ в формате:

* номер задания;
* код с пояснениями;
* результат запроса (в виде таблицы);
* общий вывод в конце по результатам анализа рынка труда.

|<center> city    | <center> Candidate | <center> timetable <br> candidate_timetable_type </center>|
| -- | -- | -- |
|![ ](data/city.png "city") |![ ](data/candidate.png "candidate")| ![](data/timetable_type.png "timetable") <br> <br> ![](data/candidate_timetable_type.png "timetable") | 

In [1]:
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Импортируем данные для соединения с SQL-сервером
import credentials as CR

In [2]:
connection = CR.connection

## 1. Предварительный анализ данных

Отобразим признаки каждой из таблиц.

In [3]:
query = f'''
SELECT
    *
FROM
    hh.CANDIDATE
LIMIT 2
'''
display(pd.read_sql_query(query, connection))

,gender,age,desirable_occupation,city_id,employment_type,current_occupation,updated_at,id,salary
0,M,32,"Oracle PL/SQL devepoler,Ведущий IT специалист,...",225,полная занятость,Ведущий IT Специалист,2019-04-26,14052,85000.0
1,M,37,Ведущий Специалист/ Эксперт,26,полная занятость,Специалист по управлению процессами и системой...,2019-04-26,27436,80000.0


In [139]:
query = f'''
SELECT DISTINCT
    *
FROM
    hh.city
ORDER BY 1 DESC
LIMIT 2
'''
display(pd.read_sql_query(query, connection))

,title,id
0,Яшалта,984
1,Яхрома,983


In [65]:
query = f'''
SELECT
    *
FROM
    hh.timetable_type
'''
display(pd.read_sql_query(query, connection))

,id,title
0,1,гибкий график
1,2,полный день
2,3,сменный график
3,4,вахтовый метод
4,5,удаленная работа


In [77]:
query = f'''
SELECT
    *
FROM
    hh.CANDIDATE_TIMETABLE_TYPE
LIMIT 2
'''
display(pd.read_sql_query(query, connection))

,id,candidate_id,timetable_id
0,9481,14473,2
1,9482,14563,1


## 2. Разведывательный анализ данных 

2.1 Рассчитайте максимальный возраст (max_age) кандидата в таблице.

In [12]:
query = f'''
SELECT 
    MAX(age) as max_age
FROM
    hh.candidate
LIMIT 5
'''
display(pd.read_sql_query(query, connection))

,max_age
0,100


2.2 Теперь давайте рассчитаем минимальный возраст (min_age) кандидата в таблице.

In [13]:
query = f'''
SELECT 
    MIN(age) as min_age
FROM
    hh.candidate
LIMIT 5
'''
display(pd.read_sql_query(query, connection))

,min_age
0,14


2.3 Попробуем «почистить» данные. Напишите запрос, который позволит посчитать для каждого возраста (age) сколько (cnt) человек этого возраста у нас есть.
Отсортируйте результат по возрасту в обратном порядке.

In [15]:
query = f'''
SELECT
    age AS "Возраст кандидата",
    COUNT(id)
FROM
    hh.candidate
GROUP BY 1
ORDER BY 1 DESC
'''
display(pd.read_sql_query(query, connection))

,Возраст кандидата,count
0,100,1
1,77,1
2,76,1
3,73,4
4,72,3
...,...,...
58,18,61
59,17,14
60,16,4
61,15,2


2.4 По данным Росстата, средний возраст занятых в экономике России составляет 39.7 лет. Мы округлим это значение до 40. Найдите количество кандидатов, которые старше данного возраста. Не забудьте отфильтровать «ошибочный» возраст 100.

In [30]:
query = f'''
SELECT
    COUNT(c.id) AS "Количество кандидатов старше 40"
FROM
    hh.candidate AS c
WHERE c.age != 100 and
c.age > 40
/*GROUP BY 1*/
ORDER BY 1 DESC
'''
display(pd.read_sql_query(query, connection))

,Количество кандидатов старше 40
0,6263


# 3. Глобальный анализ показателей
---

3.1 Для начала напишите запрос, который позволит узнать, сколько (cnt) у нас кандидатов из каждого города (city).
Формат выборки: city, cnt.
Группировку таблицы необходимо провести по столбцу title, результат отсортируйте по количеству в обратном порядке.

In [18]:
query = f'''
SELECT
    ct.title AS "Город",
    COUNT(c.city_id) AS "Количество кандидатов"
FROM
    hh.candidate AS c
JOIN 
    hh.city AS ct ON c.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
GROUP BY 1
ORDER BY 2 DESC
'''
display(pd.read_sql_query(query, connection))

,Город,Количество кандидатов
0,Москва,16622
1,Санкт-Петербург,4937
2,Краснодар,1066
3,Новосибирск,958
4,Казань,872
...,...,...
979,Адыгейск,1
980,Томилино,1
981,Черняховск,1
982,Магас,1


3.2 Москва бросается в глаза как, пожалуй, самый активный рынок труда. Напишите запрос, который позволит понять, каких кандидатов из Москвы устроит «проектная работа».
Формат выборки: gender, age, desirable_occupation, city, employment_type.
Отсортируйте результат по id кандидата.

In [24]:
query = f'''
SELECT
    cand.gender, /* пол из таблицы candidate */
    cand.age,   /* возраст из таблицы candidate */
    cand.desirable_occupation, /* желаемая позиция из таблицы candidate */
    ct.title, /* город из таблицы city */
    cand.employment_type /* тип желаемой занятости из таблицы candidate */
    
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
WHERE
    ct.title = 'Москва' AND cand.employment_type LIKE '%проектная работа%'
ORDER BY cand.id ASC
'''
display(pd.read_sql_query(query, connection))

,gender,age,desirable_occupation,title,employment_type
0,M,38,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Москва,"частичная занятость, проектная работа, полная ..."
1,M,31,Специалист,Москва,"частичная занятость, проектная работа, полная ..."
2,F,42,"pre-sale инженер, pre-sale менеджер",Москва,"частичная занятость, проектная работа, полная ..."
3,M,49,Дежурный администратор,Москва,"частичная занятость, проектная работа, полная ..."
4,M,29,Главный инженер проекта,Москва,"частичная занятость, проектная работа, полная ..."
...,...,...,...,...,...
2945,M,28,Администратор баз данных,Москва,"частичная занятость, проектная работа, полная ..."
2946,M,29,Менеджер по работе с клиентами (без поиска),Москва,"частичная занятость, проектная работа, полная ..."
2947,F,25,"Email-маркетолог, Автор",Москва,"проектная работа, полная занятость"
2948,M,26,Начальник отдела,Москва,"стажировка, волонтерство, частичная занятость,..."


In [142]:
query = f'''
SELECT
    cand.gender, /* пол из таблицы candidate */
    cand.age,   /* возраст из таблицы candidate */
    cand.desirable_occupation, /* желаемая позиция из таблицы candidate */
    ct.title, /* город из таблицы city */
    cand.employment_type /* тип желаемой занятости из таблицы candidate */
    
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
WHERE
    ct.title = 'Москва' AND cand.employment_type NOT LIKE '%проектная работа%'
ORDER BY cand.id ASC
'''
display(pd.read_sql_query(query, connection))

,gender,age,desirable_occupation,title,employment_type
0,M,29,Технический специалист,Москва,полная занятость
1,M,46,Руководитель ИТ-проектов,Москва,полная занятость
2,M,29,Инженер АСУ ТП,Москва,полная занятость
3,M,29,Ревизор,Москва,полная занятость
4,M,34,"Менеджер по работе с клиентами, Pre-sale менед...",Москва,полная занятость
...,...,...,...,...,...
13667,M,28,Сервисный инженер,Москва,"частичная занятость, полная занятость"
13668,M,23,Технический специалист со знанием нем/англ языков,Москва,полная занятость
13669,M,31,Руководитель IT-отдела,Москва,полная занятость
13670,M,24,Контент-менеджер,Москва,"частичная занятость, полная занятость"


## <center>city 
<center> <img src = "data/city.png"> </center>

## <center>candidate
<center> <img src = "data/candidate.png"> </center>

3.3 Данных оказалось многовато. Отфильтруйте только самые популярные IT-профессии — разработчик, аналитик, программист.
Обратите внимание, что данные названия могут быть написаны как с большой, так и с маленькой буквы.
Отсортируйте результат по id кандидата.

In [46]:
query = f'''
SELECT
    cand.gender, /* пол из таблицы candidate */
    cand.age,   /* возраст из таблицы candidate */
    cand.desirable_occupation, /* желаемая позиция из таблицы candidate */
    ct.title, /* город из таблицы city */
    cand.employment_type /* тип желаемой занятости из таблицы candidate */
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
WHERE
    ct.title = 'Москва' 
    AND cand.employment_type LIKE '%проектная работа%' 
    AND (lower(cand.desirable_occupation) LIKE '%разработчик%' 
    OR   lower(cand.desirable_occupation) LIKE '%аналитик%' 
    OR   lower(cand.desirable_occupation) LIKE '%программист%')
ORDER BY cand.id
'''
display(pd.read_sql_query(query, connection))

,gender,age,desirable_occupation,title,employment_type
0,M,38,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Москва,"частичная занятость, проектная работа, полная ..."
1,M,22,Программист С++,Москва,"проектная работа, частичная занятость"
2,M,25,Frontend-разработчик,Москва,"стажировка, волонтерство, частичная занятость,..."
3,M,30,Программист,Москва,"частичная занятость, проектная работа"
4,M,35,Ruby / Rails разработчик,Москва,"частичная занятость, проектная работа, полная ..."
...,...,...,...,...,...
773,M,23,junior dev/разработчик-стажер,Москва,"проектная работа, стажировка, частичная занятость"
774,M,38,"Инженер-программист, Системный администратор, ...",Москва,"проектная работа, частичная занятость, полная ..."
775,M,30,Инженер-программист,Москва,"стажировка, частичная занятость, проектная раб..."
776,M,22,Программист .NET junior,Москва,"частичная занятость, проектная работа, полная ..."


3.4 Для общей информации попробуйте выбрать номера и города кандидатов, у которых занимаемая должность совпадает с желаемой.
Формат выборки: id, city.
Отсортируйте результат по городу и id кандидата.

In [53]:
query = f'''
SELECT 
    cand.id, /* id из таблицы candidate */
    ct.title AS city   /* id города из таблицы candidate */
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */
WHERE
    cand.desirable_occupation LIKE cand.current_occupation 
ORDER BY 2,1
'''
display(pd.read_sql_query(query, connection))

,id,city
0,2009,Абакан
1,10340,Абакан
2,14449,Абакан
3,20261,Абакан
4,13705,Агрыз
...,...,...
5099,39508,Ярославль
5100,40481,Ярославль
5101,41970,Ярославль
5102,1462,Ясногорск


Распределение кандидатов, желающих изменить профессию, по городам

In [3]:
query = f'''
(SELECT 
    ct.title AS city,   /* id города из таблицы candidate */
    count(*)
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */
WHERE
    cand.desirable_occupation LIKE cand.current_occupation 
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5)
UNION ALL
(SELECT 
    'Total',  /*считаем общее количество кандидатов*/
    COUNT(*)    /*считаем строки*/
FROM
    hh.candidate AS cand
WHERE
    cand.desirable_occupation LIKE cand.current_occupation 
GROUP BY 1
ORDER BY 2 DESC)
'''
display(pd.read_sql_query(query, connection))

,city,count
0,Москва,1855
1,Санкт-Петербург,527
2,Краснодар,118
3,Новосибирск,118
4,Казань,102
5,Total,5104


3.5 Определите количество кандидатов пенсионного возраста.
Пенсионный возраст для мужчин наступает в 65 лет, для женщин — в 60 лет.

In [62]:
query = f'''
SELECT
    COUNT(cand.id) /* id из таблицы candidate */
FROM
    hh.candidate AS cand
WHERE
    (cand.gender LIKE 'F' AND cand.age >= 60 AND cand.age != 100) OR
    (cand.gender LIKE 'M' AND cand.age >= 65 AND cand.age != 100)

'''
display(pd.read_sql_query(query, connection))

,count
0,75


## 4. Анализ кандидатов для заказчиков
---
4.1 Для добывающей компании нам необходимо подобрать кандидатов из Новосибирска, Омска, Томска и Тюмени, которые готовы работать вахтовым методом.
Формат выборки: gender, age, desirable_occupation, city, employment_type, timetable_type.
Отсортируйте результат по городу и номеру кандидата.

![ ](data/interrelation.png "interrelation")

In [90]:
query = f'''
SELECT
    cand.gender, /* пол из таблицы candidate */
    cand.age,   /* возраст из таблицы candidate */
    cand.desirable_occupation, /* желаемая позиция из таблицы candidate */
    ct.title AS city, /* город из таблицы city */
    cand.employment_type, /* тип желаемой занятости из таблицы candidate */
    tt.title AS timetable_type /* желаемый тип занятости из таблицы timetable_type */
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
JOIN 
    hh.candidate_timetable_type AS ctt ON cand.id = ctt.candidate_id  /*соответствие таблиц candidate_timetable_type и hh.candidate по признаку  candidate_id */
JOIN 
    hh.timetable_type AS tt ON tt.id = ctt.timetable_id  
WHERE
    (ct.title in ('Новосибирск', 'Омск', 'Томск', 'Тюмень') 
    and tt.title = 'вахтовый метод')
ORDER BY 4, cand.id
'''
display(pd.read_sql_query(query, connection))

,gender,age,desirable_occupation,city,employment_type,timetable_type
0,M,29,ИТ Инженер,Новосибирск,полная занятость,вахтовый метод
1,M,25,Заместитель начальника лаборатории,Новосибирск,"проектная работа, стажировка, частичная занято...",вахтовый метод
2,M,30,"Ведущий инженер, Специалист по защите информации,",Новосибирск,"частичная занятость, полная занятость",вахтовый метод
3,M,23,Программист,Новосибирск,полная занятость,вахтовый метод
4,M,35,"Инженер АСУТП, инженер-электроник",Омск,полная занятость,вахтовый метод
5,M,25,Тестировщик ПО,Омск,"стажировка, полная занятость",вахтовый метод
6,M,26,Специалист технической поддержки,Томск,"частичная занятость, полная занятость",вахтовый метод
7,M,30,Менеджер проектов,Томск,"проектная работа, частичная занятость, полная ...",вахтовый метод
8,M,42,Инженер,Томск,"проектная работа, частичная занятость, полная ...",вахтовый метод
9,M,31,Инженер связи,Тюмень,полная занятость,вахтовый метод


4.2 Для заказчиков из Санкт-Петербурга нам необходимо собрать список из 10 желаемых профессий кандидатов из того же города от 16 до 21 года (в выборку включается 16 и 21, сортировка производится по возрасту) с указанием их возраста, а также добавить строку Total с общим количеством таких кандидатов. Напишите запрос, который позволит получить выборку вида:

![ ](data/task_4.2.png "task")

In [131]:
query = f'''
(SELECT
    cand.desirable_occupation, /* желаемая позиция из таблицы candidate */
    cand.age   /* возраст из таблицы candidate */
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */ 
WHERE
    (ct.title = 'Санкт-Петербург') AND (cand.age BETWEEN 16 and 21) 
ORDER BY 2
LIMIT 10)

UNION ALL
(SELECT 
    'Total',  /*считаем общее количество кандидатов*/
    COUNT(*)    /*считаем строки*/
FROM
    hh.candidate AS cand
JOIN 
    hh.city AS ct ON cand.city_id = ct.id  /*соответствие таблиц hh.candidate и hh.city по признаку city_id */  
WHERE
    (ct.title = 'Санкт-Петербург') AND (cand.age BETWEEN 16 and 21))
'''
display(pd.read_sql_query(query, connection))

,desirable_occupation,age
0,Системный администратор,16
1,Junior Разработчик C++/C#,18
2,Программист,18
3,Junior Data Scientist,18
4,Руководитель web-разработки,18
5,Специалист по IT,18
6,Unity3D developer Junior/middle,18
7,HTML-верстальщик,18
8,3D-дизайнер,18
9,Java-разработчик,18


In [154]:
query = f'''
(SELECT 
cand_data.desirable_occupation, 
cand_data.age 
from hh.candidate as cand_data 
join hh.city city_data on cand_data.city_id = city_data.id 
where city_data.title = 'Санкт-Петербург' and age between 16 and 21 
order by 2 
)

union all
(select 'Total', 
count(*) 
from hh.candidate as cand_data 
join hh.city city_data on cand_data.city_id = city_data.id 
where city_data.title = 'Санкт-Петербург' and age between 16 and 21
LIMIT 25)
'''
display(pd.read_sql_query(query, connection))

,desirable_occupation,age
0,Системный администратор,16
1,Программист,18
2,Unity3D developer Junior/middle,18
3,Java-разработчик,18
4,HTML-верстальщик,18
...,...,...
157,Ищу неофициальное трудоустройство,21
158,Оператор БД,21
159,Администратор на удаленную работу,21
160,Аналитик,21
